In [9]:
cd ..

/Users/nicolacecere/GitHub/UPM-Master/Massively-Parallel-ML


In [4]:
import os
import sys

import matplotlib.pyplot as plt
from kmeans import parallelKMeans
from pyspark import SparkContext
from read import parallelReadFile


def plot_centroids(centroids, image_size=(28, 28)):
    """
    Plot each centroid as an image.

    :param centroids: numpy array, centroids with each row representing a centroid.
    :param image_size: tuple, the (height, width) of the images represented by the centroids.
    """
    for i, centroid in enumerate(centroids):
        # Reshape the centroid to the original image dimensions
        image = centroid.reshape(image_size)

        # Plot the image
        plt.figure()
        plt.imshow(image, cmap="gray")
        plt.title(f"Centroid {i+1}")
        plt.show()

In [6]:
current_directory

'/Users/nicolacecere/GitHub/UPM-Master/Massively-Parallel-ML/MNIST/parallel_version'

In [ ]:
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

sc = SparkContext.getOrCreate()
current_directory = os.getcwd()
sc.addPyFile(current_directory + "/MNIST/parallel_version/" + "read.py")
sc.addPyFile(current_directory + "/MNIST/parallel_version/" + "kmeans.py")
# read data
data = parallelReadFile("MNIST/data/tot_mnist_shuf.csv")
data_cache = data.cache()
centroids = parallelKMeans(data_cache, 10, 10)
plot_centroids(centroids)

23/12/06 21:02:57 WARN SparkContext: The path /Users/nicolacecere/GitHub/UPM-Master/Massively-Parallel-ML/MNIST/parallel_version/read.py has been added already. Overwriting of added paths is not supported in the current version.
23/12/06 21:02:57 WARN SparkContext: The path /Users/nicolacecere/GitHub/UPM-Master/Massively-Parallel-ML/MNIST/parallel_version/kmeans.py has been added already. Overwriting of added paths is not supported in the current version.


Py4JJavaError: An error occurred while calling o43.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/nicolacecere/GitHub/UPM-Master/Massively-Parallel-ML/MNIST/parallel_version/MNIST/data/tot_mnist_shuf.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287

23/12/07 00:00:39 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 927787 ms exceeds timeout 120000 ms
23/12/07 00:00:39 WARN SparkContext: Killing executors is not supported by current scheduler.
23/12/07 00:00:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$